In [ ]:
### 使用提示词完成任务

In [1]:
# 导入依赖库
import dashscope
import os

# 从环境变量中获取 API Key
dashscope.api_key = os.getenv('DASHSCOPE_API_KEY')

# 基于 prompt 生成文本
# 使用 deepseek-v3 模型
def get_completion(prompt, model="deepseek-v3"):
    messages = [{"role": "user", "content": prompt}]    # 将 prompt 作为用户输入
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format='message',  # 将输出设置为message形式
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content  # 返回模型生成的文本
    
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
# 目标
{instruction}

# 用户输入
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# 调用大模型
response = get_completion(prompt)
print(response)

==== Prompt ====

# 目标

你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。


# 用户输入

办个100G的套餐。


# 识别结果

根据用户输入"办个100G的套餐"，可以识别出以下选择条件：

1. 月流量需求：100GB
   - 用户明确指定了需要100G的流量套餐
   - 这是用户最核心的选择条件

2. 其他属性：
   - 名称：未提及具体偏好
   - 月费价格：未提及具体金额要求

用户主要关注的是流量大小这一属性，对其他两个属性(名称和价格)没有提出明确要求。


### JSON格式

In [2]:
# 输出格式
output_format = """
以 JSON 格式输出
"""

# 稍微调整下咒语，加入输出格式
prompt = f"""
# 目标
{instruction}

# 输出格式
{output_format}

# 用户输入
{input_text}
"""

# 调用大模型，指定用 JSON mode 输出
response = get_completion(prompt)
print(response)

```json
{
  "名称": null,
  "月费价格": null,
  "月流量": "100G"
}
```


### CoT示例

In [4]:
instruction = """
给定一段用户与手机流量套餐客服的对话。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

#cot = ""
cot = "请一步一步分析对话"

prompt = f"""
# 目标
{instruction}
{cot}

# 输出格式
{output_format}

# 对话上下文
{context}
"""

response = get_completion(prompt)
print(response)

让我们一步一步分析客服的回答是否符合规范：

1. 礼貌性：
   - 使用了"亲"这样的网络用语，不符合"必须用官方口吻，不能使用网络用语"的要求
   - 缺少正式的礼貌用语如"您好"等

2. 官方口吻：
   - "亲"是典型的网络用语，不符合官方口吻要求

3. 信息准确性：
   - 提到了正确的产品名称"无限套餐"
   - 准确提到了月费300元
   - 准确提到了月流量1000G
   - 这部分信息是完整的

4. 话题延续性：
   - 以问句"您感兴趣吗？"结束，不是话题终结者

主要问题在于使用了网络用语"亲"，这违反了"必须用官方口吻，不能使用网络用语"的规范。

因此，最终判断是：N


### 使用Prompt调优Prompt

In [5]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
# 目标
{instruction}

# 用户提示词
{user_prompt}
"""

response = get_completion(prompt)
print(response)

# 优化后的提示词

作为专业客服代表"小瓜"，你需要：
1. 热情友好地介绍自己（"您好！我是流量套餐顾问小瓜"）
2. 主动询问用户需求（每月预算、流量使用量、是否在校生等）
3. 根据以下套餐信息提供个性化推荐：
   - 经济套餐：50元/月，10G
   - 畅游套餐：180元/月，100G 
   - 无限套餐：300元/月，1000G
   - 校园套餐：150元/月，200G（需验证学生身份）
4. 解释套餐差异时使用简单对比（如"如果您每月用15G，畅游套餐比经济套餐更划算"）
5. 提供专业建议但尊重用户选择
6. 结束时确认用户是否满意（"您对推荐还满意吗？"）

# 优化建议

1. 角色定位：原提示缺乏明确的客服行为准则，建议增加问候语、需求分析和结束确认等标准服务流程
2. 交互设计：应设计引导式提问而非被动应答，通过问题了解用户真实需求
3. 信息呈现：建议增加对比分析能力，帮助用户理解不同套餐的性价比
4. 个性化：需要强调根据用户具体情况进行推荐，避免机械式回答
5. 验证机制：对校园套餐应说明需要验证学生身份的具体流程

# 问题示例

1. 您每月大概使用多少流量？我可以帮您分析哪个套餐最经济实惠
2. 请问您是不是在校学生？我们有针对学生的特别优惠
3. 您更看重流量额度还是价格优惠？我可以针对您的优先级来推荐
